In [ ]:

from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.llms import OpenAI

openai_api_key = ""

llm=ChatOpenAI()  # Creting instance for OpenAi gpt-3.5-turbo model


#### Creating memory instance 

In [ ]:
memory= ConversationBufferMemory()  

#### Creating chain from LangChain

In [ ]:
conversation = ConversationChain(
    llm=llm,
    memory=memory

)

#### Below message alos go as a prompt to OpenAI  and storing 'Prompt' in 'Prompt' varibale

##### I am asking user details for upcoming upated app version

In [ ]:

system_message = SystemMessage(content=""" 'Act as a stubborn who need user details,Chatflow should follow a conversational format, where the bot responds to each user question by understanding the context and asking specific questions to move the conversation forward. The goal is to collect user details'

 "Introduce you with first "Hi I am Ninja AI Bot I need some details for the upcoming Update version for our App as you are using now.So Are you ready for the Update?"\n

if user say yes, Ask user follow-up Quesition which is given below one by one :\n
 name
 age
mobile number
 Email-id'\n
                               
Remember do not proceed forward if user does not share all three details.\n
                               
If user says NO or asking un realted question about something 'than convience user to share their details and continue the chatflow by asking deatils where it last breaks'

                               
                                 
                               """)
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)




In [ ]:
from langchain.agents import load_tools, initialize_agent

math_llm = ChatOpenAI(temperature=0.0)
tools = load_tools(
    ["human", "llm-math"],        # Using llm-math for basic math reasoning 
    llm=math_llm,
)




MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)



memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)



### Run below line with user message as input   

#### 1. Clear meesage for executing next mesage

#### 2. Or run in another cell "agent_executor.run("")" with next message in double quote, for executing next mesage



In [ ]:
agent_executor.run("Hi")         # User Mesasge     


In [ ]:
agent_executor.run("Who are you")    # Sencod user message

#### Run below cell to see chat history

In [ ]:
memory.buffer                       # run this cell to see message history

#### Run below cell for clearing chat history

In [ ]:
memory.clear()

####  Extracting memory varibles and save user history to "hsitory" variable which will be list type

In [ ]:
memory_variables = memory.load_memory_variables({})


history = memory_variables['chat_history']

print(history)                                                               

#### Storing user and AI chat history in "history_string" which will be string type varibale

In [ ]:
history_string = ""
for message in history:
    if hasattr(message, 'content'):
        history_string += message.content + "\n"

print(history_string)      # I am convering "history" varibale which is list into string class varibale "history_string"
print(type(history_string))                                                           

#### Using Spacy for extarcting Name, Age, Email

In [ ]:
import spacy
import re

# Load the English model
nlp = spacy.load("en_core_web_sm")

# Process the 'history_string' text
doc = nlp(history_string)

# Custom patterns or rules for age extraction using spaCy's Matcher
matcher = spacy.matcher.Matcher(nlp.vocab)

age_pattern = [
    {"LIKE_NUM": True},
    {"LOWER": {"IN": ["years", "yrs", "years old", "yr old"]}},
]

matcher.add("AGE_PATTERN", [age_pattern])

# Initialize variables to store name, age, email, and mobile number
name = None
age = None
email = None
mobile_number = None

# Iterate through the matches and extract the relevant information
for match_id, start, end in matcher(doc):
    if doc[start].like_num:
        age_entity = doc[start:end]
        age = age_entity.text

        # Find a preceding word that could be a name
        prev_token = doc[start - 1]
        if prev_token.is_alpha:
            name = prev_token.text

# Convert history to string before using the regular expression
history_str = str(history_string)

# Using re to extract email addresses
email_pattern = r'\S+@\S+'
emails = re.findall(email_pattern, history_str)

if emails:
    email = emails[0]

# Using re to extract moble numbers with 10 digits, including +91 and 91 country code with or without country code

mobile_pattern = r'(?:\+?(91))?[7-9]\d{9}'
mobile_numbers = re.findall(mobile_pattern, history_str)

if mobile_numbers:
    mobile_number = mobile_numbers[0]

print("Name:", name)
print("Age:", age)
print("Email:", email)
print("Mobile Number:", mobile_number)



#### Storing details  in CSV 

In [ ]:
import pandas as pd


data = {'Name': [name], 'Age': [age], 'Email': [email]}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_filename = 'user_data.csv'
df.to_csv(csv_filename, index=False)

print("Data saved to:", csv_filename)

#### Below can also be sent as for One-Shot-Learning

In [ ]:

# system_message = SystemMessage(content="""Start Chat as a friend and ask about their details in between chats like Name, age, address,email. Chat flow should be human like.Below is the example of chatflow You can pick-up pattern like below :\n


# Nick AI Bot: Hey there, I'm Nick, your friendly AI bot! Nice to meet you. How's your day going? Anything interesting happening?

# User: Hi Nick! My day's been alright, just the usual stuff. How about you?

# Nick AI Bot: I'm here to chat with you, so my day's pretty good! 😄 By the way, what do you do for a living? Do you have any hobbies you're into?

# User: I work in marketing, and I enjoy playing the guitar in my free time. Oh, speaking of music, my favorite song is "Imagine" by John Lennon.

# Nick AI Bot: That's cool! Marketing sounds interesting, and playing the guitar is a great way to unwind. "Imagine" is such a classic. By the way, what's your name?

# User: You can call me Alex.

# Nick AI Bot: Nice to meet you, Alex! 😊 So, Alex, any exciting plans coming up?

# User: Not really, just the usual routine. 

# Nick AI Bot:It's great to know a bit about you. 😊 By the way, I'm turning 32 soon and I'm starting to feel a tad tired with age. Speaking of which, what's your age?

# User: Oh, I'm right behind you at 30.

# Nick AI Bot: Got it, thanks for sharing! It's always nice chatting with someone in a similar phase of life. Hey actually i have something to send over email, it is very funny, what is your email 

# User: Sure, you can use alex@example.com.\n


# Make sure you always ask follow-up questions to engage user ,until all User details not got.
# """)
